In [1]:
import pandas as pd
import numpy as np
import os
import requests
import base64
import csv
import pickle

In [2]:
train_dir = r'Cars Dataset\train'
test_dir = r'Cars Dataset\test'

In [3]:
print("รายชื่อไฟล์ในโฟลเดอร์ test:", os.listdir(test_dir))
print("รายชื่อไฟล์ในโฟลเดอร์ train:",  os.listdir(train_dir))


รายชื่อไฟล์ในโฟลเดอร์ test: ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce', 'Swift', 'Tata Safari', 'Toyota Innova']
รายชื่อไฟล์ในโฟลเดอร์ train: ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce', 'Swift', 'Tata Safari', 'Toyota Innova']


In [4]:
brands = {
        0:"Audi",
        1:"Hyundai Creta",
        2:"Mahindra Scorpio",
        3:'Rolls Royce',
        4:'Swift',
        5:'Tata Safari',
        6:'Toyota Innova'
        }

In [5]:
def getModelNumber(model_name):
    for number, name in brands.items():
        if name == model_name:
            return number
    return None 

In [6]:
def GETHOG(part_dir):
    listData = [] 
    Amount = 0
    
    if not os.path.exists(part_dir):
        print("โฟลเดอร์ไม่มีอยู่")
        exit()

    for filebrand in os.listdir(part_dir):
        count = 0

        file_path = os.path.join(part_dir, filebrand)
        for image_filename in os.listdir(file_path):

            image_path = os.path.join(file_path, image_filename)

            with open(image_path, "rb") as image_file:
                
                image_data = image_file.read()
                base64_encoded = base64.b64encode(image_data).decode("utf-8")

                url = r'http://localhost:8080/api/gethog'
                data64 = {
                    "img": 'data:image/' + str(os.path.splitext(image_path)[1]) + ';base64,' + base64_encoded
                }

                headers = {"Content-Type": "application/json"}
                response = requests.get(url, json=data64, headers=headers)
                
                if response.status_code == 200:
                    count = count + 1
                    data = {
                        "HOG":  response.json()["Hog"],
                        "Class": getModelNumber(filebrand)
                    }
                    
                    listData.append(data) 
                else:
                    print("Error:", response.status_code)
        print(str((filebrand))+'  '+filebrand+"  "+str(count))
        Amount = Amount + count
    print(f'Amount {Amount}')
    return listData

In [7]:
list_train = GETHOG(train_dir)
list_test = GETHOG(test_dir)

Audi  Audi  814
Hyundai Creta  Hyundai Creta  271
Mahindra Scorpio  Mahindra Scorpio  316
Rolls Royce  Rolls Royce  311
Swift  Swift  424
Tata Safari  Tata Safari  441
Toyota Innova  Toyota Innova  775
Amount 3352
Audi  Audi  199
Hyundai Creta  Hyundai Creta  67
Mahindra Scorpio  Mahindra Scorpio  75
Rolls Royce  Rolls Royce  74
Swift  Swift  102
Tata Safari  Tata Safari  106
Toyota Innova  Toyota Innova  190
Amount 813


In [8]:
def save_pkmax(namesFile, listDatas):
    pkmax_filename = namesFile + ".pkmax"
    with open(pkmax_filename, "wb") as pkmax_file:
        pickle.dump(listDatas, pkmax_file)

def load_pkmax(namesFile):
    pkmax_filename = namesFile + ".pkmax"
    with open(pkmax_filename, "rb") as pkmax_file:
        listDatas = pickle.load(pkmax_file)
    return listDatas

In [9]:
save_pkmax("data_train",list_train)
save_pkmax("data_test",list_test)

In [10]:
data_train_pk = load_pkmax("data_train")
data_test_pk = load_pkmax("data_test")

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import tree

In [61]:
len(data_train_pk)-1

3351

In [62]:
def getliest_convers(data_pk,spiltName):
    data = []
    for i in range (0,len(data_pk)-1):
        da = data_pk[i][str(spiltName)]
        data.append(da)
    return data

In [63]:
X_train = getliest_convers(data_train_pk,"HOG")
Y_train = getliest_convers(data_train_pk,"Class")

In [64]:
X_test = getliest_convers(data_test_pk,"HOG")
Y_test = getliest_convers(data_test_pk,"Class")

In [65]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,Y_train)

In [66]:
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import  accuracy_score   
y_model_prediction = clf.predict(X_train)

# print(accuracy_score(Y_test,y_model_prediction)*100)
# print(confusion_matrix(Y_test,y_model_prediction))

In [69]:
print(accuracy_score(Y_test,y_model_prediction)*100)

ValueError: Found input variables with inconsistent numbers of samples: [812, 3351]